# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [15]:
# Load the libraries as required.
import os
import pandas as pd
import requests
import zipfile as zip
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge 
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import shap

c:\Users\chiny\anaconda3\envs\dsi_participant\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Create the directory
os.makedirs("C:\\Users\\chiny\\OneDrive\\Documents\\DSI\\production\\05_src\\data\\fires", exist_ok=True)

In [3]:
# Define the URL of the ZIP file to be downloaded
url = "https://archive.ics.uci.edu/static/public/162/forest+fires.zip"

# Define the local directory and ZIP file path
dir_path = "C:\\Users\\chiny\\OneDrive\\Documents\\DSI\\production\\05_src\\data\\fires"
zip_file_path = os.path.join(dir_path, "forestfires.zip")

# Download the ZIP file
response = requests.get(url)
with open(zip_file_path, 'wb') as file:
    file.write(response.content)

print(f"File downloaded and saved to {zip_file_path}")


File downloaded and saved to C:\Users\chiny\OneDrive\Documents\DSI\production\05_src\data\fires\forestfires.zip


In [4]:
# Extract the ZIP file into the specified directory
with zip.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(dir_path)

print(f"Files extracted to {dir_path}")


Files extracted to C:\Users\chiny\OneDrive\Documents\DSI\production\05_src\data\fires


In [ ]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = pd.read_csv ('C:\\Users\\chiny\\OneDrive\\Documents\\DSI\\production\\05_src\\data\\fires\\forestfires.csv', header = 0)
fires_dt.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


In [6]:
fires_dt.head()

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


In [7]:
fires_dt.describe()

,coord_x,coord_y,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,4.669246,4.299807,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292
std,2.313778,1.229900,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818
min,1.000000,2.000000,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000
25%,3.000000,4.000000,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000
50%,4.000000,4.000000,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000
75%,7.000000,5.000000,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000
max,9.000000,9.000000,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000


# Get X and Y

Create the features data frame and target data.

In [8]:
X = fires_dt.drop('area', axis=1)

In [9]:
y = fires_dt['area'] 

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer

In [11]:
std_scaler = StandardScaler()
one_hot = OneHotEncoder()
pwr_trans = PowerTransformer(method='yeo-johnson')

In [12]:
numeric_features = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
categorical_features = ['month', 'day']

In [13]:
preproc1 = ColumnTransformer(
    transformers=[
        ('num', std_scaler, numeric_features),  # Scale numbers to make them easier for models to process
        ('cat', one_hot, categorical_features)  # Turn categories into numbers
    ]
)

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [16]:
num_pipeline = Pipeline([
    ('power', pwr_trans),
    ('scaler', std_scaler)
])

preproc2 = ColumnTransformer(
    transformers=[
        ('num1', num_pipeline, numeric_features),
        ('cat', one_hot, categorical_features)  
    ]
)

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [18]:
#Regressor
Base_reg = Ridge()
Adv_reg = DecisionTreeRegressor()

In [19]:
# Pipeline A = preproc1 + baseline

pipeline_A = Pipeline([
    ('preprocessing', preproc1),  # First step: Preprocess the data
    ('regressor', Ridge())  # Second step: Apply Ridge regression to make predictions
])

In [20]:
# Pipeline B = preproc2 + baseline
pipeline_B = Pipeline([
    ('preprocessing', preproc2),  
    ('regressor', Ridge())  
])

In [21]:
# Pipeline C = preproc1 + advanced model
pipeline_C = Pipeline([
    ('preprocessing', preproc1),  
    ('regressor', DecisionTreeRegressor())
])

In [22]:
# Pipeline D = preproc2 + advanced model
pipeline_D = Pipeline([
    ('preprocessing', preproc2),  
    ('regressor', DecisionTreeRegressor())
])
    

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
ridge_params_A = {'regressor__alpha': [0.1, 1.0, 10.0]}  
ridge_params_B = {'regressor__alpha': [0.5, 10.0, 15.0]} 
tree_params_C = {'regressor__max_depth': [2, 4, 6, None]} 
tree_params_D = {'regressor__max_depth': [3, 5, 10, None]} 

ridge_grid_A = GridSearchCV(pipeline_A, ridge_params_A, cv=5, scoring='r2')
ridge_grid_B = GridSearchCV(pipeline_B, ridge_params_B, cv=10, scoring='r2')
tree_grid_C = GridSearchCV(pipeline_C, tree_params_C, cv=5, scoring='r2')
tree_grid_D = GridSearchCV(pipeline_D, tree_params_D, cv=10, scoring='r2')

ridge_grid_A.fit(X_train, y_train)
ridge_grid_B.fit(X_train, y_train)
tree_grid_C.fit(X_train, y_train)
tree_grid_D.fit(X_train, y_train)

c:\Users\chiny\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\chiny\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\model_selection\_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "c:\Users\chiny\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "c:\Users\chiny\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 371, in _score
    y_pred = method_caller(
  File "c:\Users\chiny\anaconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py", line 89, in _cached_call
    result, _ = _get_response_values(
  File "c:\Users\chin

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num1',
                                                                         Pipeline(steps=[('power',
                                                                                          PowerTransformer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor', DecisionTreeRegressor())]),
             param_grid={'regressor__max_depth': [3, 5, 10, None]},
             scoring='r2')

In [38]:
ridge_pred_A = ridge_grid_A.predict(X_test)
ridge_pred_B = ridge_grid_B.predict(X_test)
tree_pred_C = tree_grid_C.predict(X_test)
tree_pred_D = tree_grid_D.predict(X_test)

ridge_mse_A = mean_squared_error(y_test, ridge_pred_A)
ridge_r2_A = r2_score(y_test, ridge_pred_A)

ridge_mse_B = mean_squared_error(y_test, ridge_pred_B)
ridge_r2_B = r2_score(y_test, ridge_pred_B)

tree_mse_C = mean_squared_error(y_test, tree_pred_C)
tree_r2_C = r2_score(y_test, tree_pred_C)

tree_mse_D = mean_squared_error(y_test, tree_pred_D)
tree_r2_D = r2_score(y_test, tree_pred_D)


print(f"Ridge Regression A - Best Alpha: {ridge_grid_A.best_params_['regressor__alpha']}, MSE: {ridge_mse:.4f}, R²: {ridge_r2:.4f}")
print(f"Ridge Regression B - Best Alpha: {ridge_grid_B.best_params_['regressor__alpha']}, MSE: {ridge_mse:.4f}, R²: {ridge_r2:.4f}")
print(f"Decision Tree C - Best Max Depth: {tree_grid_C.best_params_['regressor__max_depth']}, MSE: {tree_mse:.4f}, R²: {tree_r2:.4f}")
print(f"Decision Tree D - Best Max Depth: {tree_grid_D.best_params_['regressor__max_depth']}, MSE: {tree_mse:.4f}, R²: {tree_r2:.4f}")

Ridge Regression A - Best Alpha: 0.1, MSE: 11578.3325, R²: 0.0178
Ridge Regression B - Best Alpha: 0.5, MSE: 11578.3325, R²: 0.0178
Decision Tree C - Best Max Depth: 2, MSE: 12382.6896, R²: -0.0505
Decision Tree D - Best Max Depth: 3, MSE: 12382.6896, R²: -0.0505


# Evaluate

+ Which model has the best performance?

# Export

+ Save the best performing model to a pickle file.

In [39]:
results = {
    'A': {'r2': r2_score(y_test, ridge_pred_A)},
    'B': {'r2': r2_score(y_test, ridge_pred_B)},
    'C': {'r2': r2_score(y_test, tree_pred_C)},
    'D': {'r2': r2_score(y_test, tree_pred_D)}
}
best_model_key = max(results.items(), key=lambda x: x[1]['r2'])[0]
best_model = {'A': ridge_grid_A, 'B': ridge_grid_B, 'C': tree_grid_C, 'D': tree_grid_D}[best_model_key]
best_model

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num1',
                                                                         Pipeline(steps=[('power',
                                                                                          PowerTransformer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['coord_x',
                                                                          'coord_y',
                                                                          'ffmc',
                                                                          'dmc',
                                                                          'dc',
                                                                          'isi',
                                                                          'temp',
                                                                          'rh',
                                                                          'wind',
                                                                          'rain']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         ['month',
                                                                          'day'])])),
                                       ('regressor', Ridge())]),
             param_grid={'regressor__alpha': [0.5, 10.0, 15.0]}, scoring='r2')

In [ ]:
preprocessed_X_test = best_model.named_steps['preprocessing'].transform(X_test)
explainer = shap.TreeExplainer(best_model.named_steps['regressor'])
shap_values = explainer.shap_values(preprocessed_X_test)

shap.summary_plot(shap_values, features=preprocessed_X_test, feature_names=X_test.columns)


In [ ]:
import pickle
import joblib
joblib.dump(best_model.best_estimator_, 'best_model.pkl')


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.